In [1]:
"""Cancelling transactions

Demonstration of how to use the CancelTransactions endpoint to cancel transactions in a LUSID portfolio.

Attributes
----------
transactions
cocoon - seed_data
holdings
cancel transactions
"""

'Cancelling transactions\n\nDemonstration of how to use the CancelTransactions endpoint to cancel transactions in a LUSID portfolio.\n\nAttributes\n----------\ntransactions\ncocoon - seed_data\nholdings\ncancel transactions\n'

# Cancelling transactions in LUSID

This notebooks shows how you can use the [<b>cancel transactions</b>](https://www.lusid.com/docs/api/#operation/CancelTransactions) endpoint to cancel transactions in a LUSID portfolio. For the pruposes of this demo, we will first seed a portfolio with 21 transactions spanning cash and various FTSE 100 stocks. We will then cancel 1 <i> Aviva</i> transaction and 2 <i> BHP </i> transactions.

### Setup LUSID

In [2]:
# Import general purpose packages
import os
import json
from datetime import datetime, timedelta
import pytz

# Import lusid specific packages
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id

# Import data wrangling packages
import pandas as pd

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# Load a mapping file for DataFrame headers for the build transaction response

with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)

# Load a mapping file for DataFrame headers for the get holdings response

with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

### 1) Load default transactions into a new scope

In [4]:
# Create a new scope

scope = create_scope_id()
portfolio_code = "EQUITY_UK"

In [5]:
# Load a file of equity transactions

transactions_file = r"data/cancels/equity_transactions.csv"
transactions_df = pd.read_csv(transactions_file)
transactions_df["portfolio_code"] = portfolio_code

In [6]:
# Load portfolios, instruments, and transactions

seed_data_response = seed_data(
    api_factory,
    ["portfolios", "instruments", "transactions"],
    scope,
    transactions_file,
    "csv",
)

In [7]:
# Define the transaction portfolio API

txn_port_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

### 2) Lets check our holdings

We have:

* 132,000 units in Aviva
* 120,000 units in BHP

In [8]:
response = txn_port_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df

,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_YN3FEUD5,{},Aviva,EQUITY_UK,3967-afda-ea4e-d0,P,132000.0,132000.0,660000.0,GBP,0.0,GBP
1,LUID_7EQ664NC,{},BHP,EQUITY_UK,3967-afda-ea4e-d0,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_F87JY45S,{},Barclays,EQUITY_UK,3967-afda-ea4e-d0,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_8X5TTJTL,{},BP,EQUITY_UK,3967-afda-ea4e-d0,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_H9EN8GYB,{},HSBC,EQUITY_UK,3967-afda-ea4e-d0,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,CCY_GBP,{},GBP,EQUITY_UK,3967-afda-ea4e-d0,B,3260000.0,3260000.0,3260000.0,GBP,0.0,GBP
6,LUID_OSXJYVK6,{},Morrisons,EQUITY_UK,3967-afda-ea4e-d0,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
7,LUID_6RGYR2L4,{},Tesco,EQUITY_UK,3967-afda-ea4e-d0,P,12000.0,12000.0,100000.0,GBP,0.0,GBP
8,LUID_2IXG0MUX,{},Rightmove,EQUITY_UK,3967-afda-ea4e-d0,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
9,LUID_OSHLMH9J,{},vodafone,EQUITY_UK,3967-afda-ea4e-d0,P,900000.0,900000.0,900000.0,GBP,0.0,GBP


### 3) What transactions make up our Aviva and BHP holdings?

In [9]:
build_transactions_response = txn_port_api.build_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_query_parameters=models.TransactionQueryParameters(
        start_date="2020-01-01", end_date="2020-12-31"
    ),
    property_keys=["Instrument/default/Name"],
)


build_transactions_df = lusid_response_to_data_frame(
    build_transactions_response,
    rename_properties=True,
    column_name_mapping=build_transactions_json_mapping,
)
build_transactions_df.head(5)

,TransactionId,TransactionType,TransactionTypeDesc,ClientId,LusidInstrumentId,TransactionDate,SettlementDate,Units,TransactionAmount,Price,PriceType,TotalConsideration,TotalConsiderationCurrency,ExchangeRate,TransactionToPortfolioRate,TransactionCurrency,strategy(3967-afda-ea4e-d0-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),ResultantHolding,InstrumentName,TransactionStatus,EntryDateTime,RealisedGainLoss,InstrumentCurrency
0,trd_0001,Buy,Purchase,EQ_1234,LUID_YN3FEUD5,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,120000.0,600000.0,5.0,Price,600000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3967-afda-ea4e-d0,120000.0,Aviva,Active,2021-03-15 11:31:08.500995+00:00,[],NaN
1,trd_0002,Buy,Purchase,EQ_1234,LUID_YN3FEUD5,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,12000.0,60000.0,5.0,Price,60000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3967-afda-ea4e-d0,132000.0,Aviva,Active,2021-03-15 11:31:08.500995+00:00,[],NaN
2,trd_0003,Buy,Purchase,EQ_1235,LUID_7EQ664NC,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,60000.0,1080000.0,18.0,Price,1080000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3967-afda-ea4e-d0,60000.0,BHP,Active,2021-03-15 11:31:08.500995+00:00,[],NaN
3,trd_0004,Buy,Purchase,EQ_1235,LUID_7EQ664NC,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,60000.0,1080000.0,18.0,Price,1080000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3967-afda-ea4e-d0,120000.0,BHP,Active,2021-03-15 11:31:08.500995+00:00,[],NaN
4,trd_0005,Buy,Purchase,EQ_1236,LUID_F87JY45S,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,ftse_tracker,EQUITY_UK,3967-afda-ea4e-d0,150000.0,Barclays,Active,2021-03-15 11:31:08.500995+00:00,[],NaN


### 4) Cancel one of the Aviva transactions

Cancel an Aviva transaction of 120,000 units

In [10]:
cancel_response = txn_port_api.cancel_transactions(
    scope=scope, code=portfolio_code, transaction_ids=["trd_0001"]
)

first_cancel_datetime = cancel_response.as_at

print(f"The first cancel datetime request is: {first_cancel_datetime}")

The first cancel datetime request is: 2021-03-15 11:31:09.641562+00:00


### 5) Cancel two of the BHP transactions

Cancel two BHP transactions, both for 60,000 units.

In [11]:
cancel_response = txn_port_api.cancel_transactions(
    scope=scope, code=portfolio_code, transaction_ids=["trd_0003", "trd_0004"]
)

second_cancel_datetime = cancel_response.as_at

print(f"The second cancel datetime request is: {first_cancel_datetime}")

The second cancel datetime request is: 2021-03-15 11:31:09.641562+00:00


### 6) Check holdings again

We can see two updates:

* Holdings in Aviva have been reduced 
* We no longer have any BHP holdings

In [12]:
response = txn_port_api.get_holdings(
    scope=scope, code=portfolio_code, property_keys=["Instrument/default/Name"]
)


holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df

,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_YN3FEUD5,{},Aviva,EQUITY_UK,3967-afda-ea4e-d0,P,12000.0,12000.0,60000.0,GBP,0.0,GBP
1,LUID_F87JY45S,{},Barclays,EQUITY_UK,3967-afda-ea4e-d0,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
2,LUID_8X5TTJTL,{},BP,EQUITY_UK,3967-afda-ea4e-d0,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
3,LUID_H9EN8GYB,{},HSBC,EQUITY_UK,3967-afda-ea4e-d0,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
4,CCY_GBP,{},GBP,EQUITY_UK,3967-afda-ea4e-d0,B,6020000.0,6020000.0,6020000.0,GBP,0.0,GBP
5,LUID_OSXJYVK6,{},Morrisons,EQUITY_UK,3967-afda-ea4e-d0,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
6,LUID_6RGYR2L4,{},Tesco,EQUITY_UK,3967-afda-ea4e-d0,P,12000.0,12000.0,100000.0,GBP,0.0,GBP
7,LUID_2IXG0MUX,{},Rightmove,EQUITY_UK,3967-afda-ea4e-d0,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
8,LUID_OSHLMH9J,{},vodafone,EQUITY_UK,3967-afda-ea4e-d0,P,900000.0,900000.0,900000.0,GBP,0.0,GBP
9,LUID_89QAQSYC,{},Anglo American plc,EQUITY_UK,3967-afda-ea4e-d0,P,70000.0,70000.0,1400000.0,GBP,0.0,GBP


### 7) Check holdings with AsAt just before the cancellation

The portfolio still owns:

* 132,000 units Aviva
* 120,000 units BHP

In [13]:
as_at_time = first_cancel_datetime - timedelta(milliseconds=10)

print(f"The current datetime is {datetime.now(tz=pytz.UTC)}...")
print(f"Getting holdings as at {as_at_time}...")
print(
    f"This is just before the first cancellation datetime of {first_cancel_datetime}..."
)


response = txn_port_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    as_at=as_at_time,
)

holdings_df = lusid_response_to_data_frame(
    response, rename_properties=True, column_name_mapping=get_holdings_json_mapping
)

holdings_df

The current datetime is 2021-03-15 11:31:10.230580+00:00...
Getting holdings as at 2021-03-15 11:31:09.631562+00:00...
This is just before the first cancellation datetime of 2021-03-15 11:31:09.641562+00:00...


,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_YN3FEUD5,{},Aviva,EQUITY_UK,3967-afda-ea4e-d0,P,132000.0,132000.0,660000.0,GBP,0.0,GBP
1,LUID_7EQ664NC,{},BHP,EQUITY_UK,3967-afda-ea4e-d0,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_F87JY45S,{},Barclays,EQUITY_UK,3967-afda-ea4e-d0,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_8X5TTJTL,{},BP,EQUITY_UK,3967-afda-ea4e-d0,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_H9EN8GYB,{},HSBC,EQUITY_UK,3967-afda-ea4e-d0,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,CCY_GBP,{},GBP,EQUITY_UK,3967-afda-ea4e-d0,B,3260000.0,3260000.0,3260000.0,GBP,0.0,GBP
6,LUID_OSXJYVK6,{},Morrisons,EQUITY_UK,3967-afda-ea4e-d0,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
7,LUID_6RGYR2L4,{},Tesco,EQUITY_UK,3967-afda-ea4e-d0,P,12000.0,12000.0,100000.0,GBP,0.0,GBP
8,LUID_2IXG0MUX,{},Rightmove,EQUITY_UK,3967-afda-ea4e-d0,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
9,LUID_OSHLMH9J,{},vodafone,EQUITY_UK,3967-afda-ea4e-d0,P,900000.0,900000.0,900000.0,GBP,0.0,GBP
